In [1]:
import requests
from bs4 import BeautifulSoup
from db import db_connect, db_close

Election DB 스키마
- area table: aid(pk), aname(지역구 명)
- party table: pid(pk), pname(정당 명)
- mp table: mid(pk), mname(국회의원 명), party_id(forign key), area_id(forign key), win(당선 여부)

In [4]:
def get_urls():
    url = 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0'
    area_url = []
    
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawls = soup.select('table[class="nowraplinks collapsible autocollapse navbox-inner"] tbody')[6]
    crawls = crawls.select('tr')[7]
    
    for crawl in crawls.select('li a'):
        area_url.append('https://ko.wikipedia.org'+crawl.get('href'))
    
    return area_url

In [5]:
def get_area(area_url):
    stopword = ['선거통계', '선거구별 결과', '지역구', '비례대표']
    area_ls = []
    for url in area_url:
        res = requests.get(url).text
        soup = BeautifulSoup(res,'html.parser')
        crawl = soup.select('h3 span[class="mw-headline"]')
        for area in crawl:
            if area.text not in stopword:
                area_ls.append("('"+area.text+"')")
    
    return area_ls

In [6]:
db = db_connect()

In [7]:
cursor = db.cursor()

In [8]:
area_url = get_urls()
area_ls = get_area(area_url)

In [9]:
var_string = ','.join(area_ls)
sql = "INSERT INTO area (aname) VALUES %s;" % var_string

In [10]:
cursor.execute(sql)

253

In [11]:
db.commit()

In [12]:
db_close(db)